In [1]:
import operator
import math
import random

import numpy as np
from scipy import linalg

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# General configuration

In [ ]:
DIM_2D = True # 2D - True, 3D - False

In [ ]:
N = 256 # number of collocation points to calculate boundary data
Nstep = 4 # specify division step.
assert N % Nstep == 0

In [ ]:
G1_DIRICHLET_COND = BoundaryCondition(lambda X: np.zeros(shape=(X.shape[1],1), dtype=float)) ## Dirichlet condition
G2_NEUMAN_COND = BoundaryCondition(lambda X: np.ones(shape=(X.shape[1],1), dtype=float)*3) ## Neuman condition

In [ ]:
EXTERIOR_BOUNDARY_RADIUS = 5

In [ ]:
EPS = 1e-15

In [ ]:
RAND_CONST_MIN = -10
RAND_CONST_MAX = 10

In [ ]:
NUM_ISLANDS = 3
POP_SIZE = 100

In [ ]:
NOISE_LEVEL = 0.02 # 2% of noise
# TODO: add noise to boundary data

### Radial function for interior boundary

In [ ]:
if DIM_2D:
    R_EXACT = lambda t: 2 + math.sin(t)*(0.1 + math.cos(3*t))
#     R_EXACT = lambda t: 2*math.sqrt(math.cos(t)**2 + math.sin(t)**2/4)
else:
    R_EXACT = lambda theta, phi: 2 +  math.sqrt(4.25 + 3*math.cos(3*theta))
#     R_EXACT = lambda theta, phi: 2*math.sqrt(math.cos(2*theta)+ math.sqrt(2 - math.sin(2*theta)**2))

R_EXACT = np.vectorize(R_EXACT)

In [ ]:
if DIM_2D:
    collocation = Collocation2D(N)
    
    G2_BOUNDARY = Circle(collocation, EXTERIOR_BOUNDARY_RADIUS)
    G2_VALUES = G2_BOUNDARY()
    G1_BOUNDARY = StarLikeCurve(collocation)
    G1_VALUES = G1_BOUNDARY(R_EXACT(collocation.thetas)) # Values in collocation points
    
    HELPER = MFSHelper2D(collocation, G2_BOUNDARY, G1_DIRICHLET_COND, G2_NEUMAN_COND)
else:
    collocation = Collocation3D(N)
    
    G2_BOUNDARY = Sphere(collocation, EXTERIOR_BOUNDARY_RADIUS)
    G2_VALUES = G2_BOUNDARY()
    G1_BOUNDARY = StarLikeSurface(collocation)
    G1_VALUES = G1_BOUNDARY(R_EXACT(collocation.thetas, collocation.phis)) # Values in collocation points
    
    HELPER = MFSHelper3D(collocation, G2_BOUNDARY, G1_DIRICHLET_COND, G2_NEUMAN_COND)

## Find the Dirichlet data on Gamma2

In [ ]:
A = HELPER.form_matrix(G1_VALUES.copy()) # A = (2NxN)
b = HELPER.form_column(G1_VALUES.copy())
lambda_ = linalg.lstsq(A,b)[0]

In [ ]:
G2_DIRICHLET_VALUES = HELPER.uApprox(lambda_, G2_VALUES.copy(), G1_VALUES.copy())

## Recalculate data

In [ ]:
N = int(N/Nstep)

In [ ]:
if DIM_2D:
    collocation = Collocation2D(N)
    G2_BOUNDARY = Circle(collocation, EXTERIOR_BOUNDARY_RADIUS)
    G2_VALUES = G2_BOUNDARY()
    G1_BOUNDARY = StarLikeCurve(collocation)
    G1_VALUES = G1_BOUNDARY(R_EXACT(collocation.thetas)) # Values in collocation points
    HELPER = MFSHelper2D(collocation, G2_BOUNDARY, G1_DIRICHLET_COND, G2_NEUMAN_COND)
    G2_DIRICHLET_VALUES = G2_DIRICHLET_VALUES[0:len(G2_DIRICHLET_VALUES)-1:Nstep]
else:
    collocation = Collocation3D(N)
    G2_BOUNDARY = Sphere(collocation, EXTERIOR_BOUNDARY_RADIUS)
    G2_VALUES = G2_BOUNDARY()
    G1_BOUNDARY = StarLikeSurface(collocation)
    G1_VALUES = G1_BOUNDARY(R_EXACT(collocation.thetas, collocation.phis)) # Values in collocation points
    HELPER = MFSHelper3D(collocation, G2_BOUNDARY, G1_DIRICHLET_COND, G2_NEUMAN_COND)
    G2_DIRICHLET_VALUES = G2_DIRICHLET_VALUES[0:len(G2_DIRICHLET_VALUES)-1:Nstep]

# Evaluation

In [ ]:
def feasible2D(individual, thetas, curvature_threshold):
    """Feasibility function for the individual. Returns True if feasible False
    otherwise."""
    ## evaluate function values
    func_vect = np.vectorize(toolbox.compile(expr=individual))
    
    try:
#         f_vals = func_vect(thetas)*individual.scale
        f_vals = func_vect(thetas)
    except Exception as e:
        return False
    
    ## Save calculated values to evaluate the error or distance
    individual.eval_values = f_vals[:-1]
    
    ## Check if all values are finite
    if not np.isfinite(f_vals).all():
        return False
    
    ## Singular boundary
    if (abs(f_vals) < EPS).all(): 
        return False
    
    ## Interior boundary has to be inside outter boundary
    if not (abs(f_vals) < EXTERIOR_BOUNDARY_RADIUS).all():
        return False
    
    ## Check if function is periodic
    if abs(f_vals[0] - f_vals[-1]) > EPS:
        return False
    
    return True

In [ ]:
def eval2D(individual):    
    g1_approx_values = G1_BOUNDARY(individual.eval_values)

    A =HELPER.form_matrix(g1_approx_values.copy())
    b = HELPER.form_column(g1_approx_values.copy())
    
    try: # Sometimes got an error about zero division in log in formulas. Need to check if there is any error
        lambda_ = linalg.lstsq(A,b)[0]
    except:
        print(f"F:{individual},  vals: ",individual.eval_values)
        return 1e9,
    
    g2_dirichlet_approx = HELPER.uApprox(lambda_,HELPER.g2_coll,g1_approx_values)
    individual.error_values = G2_DIRICHLET_VALUES-g2_dirichlet_approx

#     individual.last_fitness = np.sum((G2_DIRICHLET_VALUES-g2_dirichlet_approx)**2)/N ## MSE error
    individual.last_fitness = np.sqrt((2*np.pi/N)*np.sum((G2_DIRICHLET_VALUES-g2_dirichlet_approx)**2)) ## L2 norm
    
    return individual.last_fitness,

In [ ]:
def feasible3D(individual, thetas, phis):
    """
    Feasibility function for the individual. Returns True if feasible False
    otherwise.
    
    thetas has shape (NxN) and next structure:
    
    t0  t1  t1  ...  tn
    t0  t1  t1  ...  tn
     .   .   .  ...  .
    t0  t1  t1  ...  tn
    
    phis has shape (NxN) and next structure:
    
    p0  p0  p0  ...  p0
    p1  p1  p1  ...  p1
     .   .   .  ...  .
    pn  pn  pn  ...  pn
    
    
    """    
    # Evaluate function values
    func_vect = np.vectorize(toolbox.compile(expr=individual))

    try:
#         f_vals = func_vect(theta = thetas,phi = phis)*individual.scale
        f_vals = func_vect(theta = thetas,phi = phis)
    except:
        return False
    
    # Save calculated values to evaluate the error or distance
    individual.eval_values = f_vals
    
    # Check if all values are finite
    if not np.isfinite(f_vals).all():
        return False
    
    # Check for singular surface
    if (abs(f_vals) < EPS).all():
        return False
    
    # Interior boundary has to be inside outter boundary
    if not (abs(f_vals) < EXTERIOR_BOUNDARY_RADIUS).all():
        return False
    
    # Check if surface is closed
    
    # r(0,phi_1) = r(0,phi_2) = ... = r(0, phi_n)
    thetas_0 = f_vals[:,0]
    if not np.isclose(thetas_0, thetas_0[0], atol=EPS).all():
        return False
    
    # r(pi,phi_1) = r(pi,phi_2) = ... = r(pi, phi_n)
    thetas_pi = f_vals[:,-1]
    if not np.isclose(thetas_pi, thetas_pi[0], atol=EPS).all():
        return False    
    
    # r(thetas,0) = r(thetas,2pi)
    if not np.allclose(f_vals[0], f_vals[-1], atol=EPS):
        return False
    
    return True

In [ ]:
def eval3D(individual):    
    g1_approx_values = G1_BOUNDARY(individual.eval_values.diagonal()) # take digagonal values (theta_0, phi_0), ..., (theta_n,phi_n)
    
#     return linalg.norm(G1_VALUES - g1_approx_values), # Test evaluation
    
    A =HELPER.form_matrix(g1_approx_values.copy())
    b = HELPER.form_column(g1_approx_values.copy())
    
    try: ## Sometimes got an error about zero division in log in formulas. Need to check if there is any error
        lambda_ = linalg.lstsq(A,b)[0]
    except:
#         print(f"F:{individual},  vals: ",individual.eval_values)
        return 1e9,
    
    g2_dirichlet_approx = HELPER.uApprox(lambda_,HELPER.g2_coll,g1_approx_values)
    
    individual.error_values = G2_DIRICHLET_VALUES-g2_dirichlet_approx
    individual.last_fitness = np.sum((G2_DIRICHLET_VALUES-g2_dirichlet_approx)**2)/N ## MSE error
    
    return individual.last_fitness,
#     return linalg.norm(G2_DIRICHLET_VALUES-g2_dirichlet_approx), 
#     return np.sqrt((2*np.pi/N)*np.sum((G2_DIRICHLET_VALUES-g2_dirichlet_approx)**2)),

# Configure deap

In [3]:
def protectedDiv(left, right):
    if abs(right) < 1e-5:
        return 1.0
    return operator.truediv(left, right)
def protectedSqrt(val):
    return math.sqrt(abs(val))

def pow2(val):
    return math.pow(val,2)

def pow3(val):
    return math.pow(val,2)

def pow4(val):
    return math.pow(val,2)

def pow5(val):
    return math.pow(val,2)

In [4]:
if DIM_2D:
    pset = gp.PrimitiveSet("MAIN", 1) # in 2d radial function has only one parameter
else:
    pset = gp.PrimitiveSet("MAIN", 2) # in 3d radial function has two parameters (phi, theta)

# some binary operators
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.mul, 2)
# pset.addPrimitive(protectedPow, 2) ## TODO: check and add
# pset.addPrimitive(math.pow, 2)


# some unary operators
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(protectedSqrt, 1)

##### For test
pset.addPrimitive(pow2, 1)
pset.addPrimitive(pow3, 1)
pset.addPrimitive(pow4, 1)
pset.addPrimitive(pow5, 1)
#####

# some useful constants
pset.addTerminal(np.pi,name='pi')
pset.addTerminal(np.e,name='e')
pset.addEphemeralConstant("randUniform", lambda: random.uniform(RAND_CONST_MIN, RAND_CONST_MAX))

# rename arguments
if DIM_2D:
    pset.renameArguments(ARG0='t')
else:
    pset.renameArguments(ARG0='theta')
    pset.renameArguments(ARG1='phi')

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual",
               gp.PrimitiveTree,
               fitness=creator.FitnessMin, 
               eval_values = np.empty(N, dtype=float),
               error_values = np.empty(N, dtype=float),
               last_fitness = float,
               scale = 1.0)

In [6]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("island", tools.initRepeat, list, toolbox.individual,POP_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.island,NUM_ISLANDS)
toolbox.register("compile", gp.compile, pset=pset)

In [ ]:
if DIM_2D:
    toolbox.register("evaluate", eval2D)
    toolbox.register("evaluate_feasible",feasible2D, thetas = collocation.thetas_closed, curvature_threshold = 25.0)
    toolbox.decorate("evaluate", tools.DeltaPenalty(toolbox.evaluate_feasible, 1e7))
else:
    thetas_mesh, phis_mesh = collocation.mesh_closed
    toolbox.register("evaluate", eval3D)
    toolbox.register("evaluate_feasible",feasible3D, thetas = thetas_mesh, phis = phis_mesh)
    toolbox.decorate("evaluate", tools.DeltaPenalty(toolbox.evaluate_feasible, 1e10)) 

## Custom selection operators

In [ ]:
from operator import attrgetter

def selectTournamentUnique(individuals, k, tournsize,best = max, fit_attr="fitness"):
    chosen = []
    while(len(chosen) < k):
        aspirants = tools.selRandom(individuals, tournsize)
        selected_best = best(aspirants, key=attrgetter(fit_attr))
        if(selected_best not in chosen):
            chosen.append(selected_best)
    return chosen

In [ ]:
toolbox.register("select_tournament", tools.selTournament, tournsize=3)
toolbox.register("select_worst", tools.selWorst)

## Custom crossover operators

In [ ]:
def chooseMate(ind1, ind2, operators, probs): # operators = [(operator,prob),...]
    op = np.random.choice(operators,1,probs)[0]
    return op(ind1, ind2)

In [ ]:
def cxUniform(ind1, ind2, swapProb):
    if len(ind1) < 2 or len(ind2) < 2:
        # No crossover on single node tree
        return ind1, ind2
    
    stack=[]
    # append left subtrees slices
    slice1,slice2=ind1.searchSubtree(1),ind2.searchSubtree(1)
    stack.append((slice1,slice2))
    
    # Append rigth subtrees slices if both exists
    if ind1.root.arity == 2 and ind2.root.arity == 2:
        stack.append((ind1.searchSubtree(slice1.stop),ind2.searchSubtree(slice2.stop)))
        
    while stack:
        # take subtrees slices
        slice1, slice2=stack.pop()
        
        if random.random() < swapProb:
            # If arity the same just swap nodes else swap subtrees
            if ind1[slice1.start].arity == ind2[slice2.start].arity:
                ind1[slice1.start], ind2[slice2.start]=ind2[slice2.start],ind1[slice1.start]
            else:
                ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]
                slice1, slice2=ind1.searchSubtree(slice1.start),ind2.searchSubtree(slice2.start)
        
        # Append subtrees slices
        if ind1[slice1.start].arity == 0 or ind2[slice2.start].arity == 0:
            continue
        elif ind1[slice1.start].arity == 1 or ind2[slice2.start].arity == 1:
            stack.append((ind1.searchSubtree(slice1.start+1),ind2.searchSubtree(slice2.start+1)))
        else:
            slice1, slice2=ind1.searchSubtree(slice1.start+1),ind2.searchSubtree(slice2.start+1)
            stack.append((slice1,slice2))
            stack.append((ind1.searchSubtree(slice1.stop), ind2.searchSubtree(slice2.stop)))
        
    return ind1,ind2

In [ ]:
toolbox.register("mate_one_point", gp.cxOnePoint)
toolbox.register("mate_uniform", cxUniform, swapProb = 0.25)

'''
 Crossover Operator |   Apply probability
--------------------|----------------------
 One point          |         60%
 Uniform            |         40% 

'''

toolbox.register("mate",chooseMate,
                operators = (toolbox.mate_one_point,
                             toolbox.mate_uniform),
                 probs = (0.6,0.4))

In [ ]:
from functools import wraps

# def resetScaleDecorator(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         ret_inds = func(*args, **kwargs)
#         for ind in ret_inds:
#             ind.scale = 1.0
#         return ret_inds

#     return wrapper

def scaleMateDecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ret_inds = func(*args, **kwargs)
        avg_scale = sum([ind.scale for ind in args])/len(ret_inds)
        for ind in ret_inds:
            ind.scale = avg_scale
        return ret_inds

    return wrapper

## Custom mutation operators

In [ ]:
def mutScale(individual, scale):
    fit = individual.last_fitness
    
    individual.scale += np.random.uniform(-scale*fit,scale*fit)
    
    return individual,

In [ ]:
def mutHoist(individual):
    '''
    This operator set randomly selected subtree as a new individual
    
    '''
    # We don't want to "shrink" the root
    if len(individual) < 3 or individual.height <= 1:
        return individual,
    
    index = random.randrange(1,len(individual)) # choose random node ( not root)
    slice_ = individual.searchSubtree(index)
    return creator.Individual(individual[slice_]),

In [ ]:
def mutEphemeralByFitness(individual, scale, mode):
    ephemerals_idx = [index
                      for index, node in enumerate(individual)
                      if isinstance(node, gp.Ephemeral)]
    
    if len(ephemerals_idx) > 0:
        fit = individual.last_fitness
        if mode == 'one':
            ephemerals_idx = (random.choice(ephemerals_idx),)
        
        for i in ephemerals_idx:
            individual[i].value += np.random.uniform(-scale*fit,scale*fit)
            
    return individual,

In [ ]:
def mutAddConstant(individual, pset):
    if len(individual) < 3 or individual.height <= 1:
        return individual,
    
    index = random.randrange(1,len(individual)) # choose random node ( not root)
    slice_ = individual.searchSubtree(index)
    mulPrimitive = gp.Primitive(operator.mul.__name__, [object, object], object)

    individual[slice_] = [mulPrimitive] + individual[slice_] + [gp.randUniform()]# Warning: function name has to be defined in pset
    return individual,

In [ ]:
def chooseMutate(individual, operators,probs): # operators = [(operator,prob),...]
    op = np.random.choice(operators, p=probs)
    return op(individual)

In [ ]:
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
# Standart ( from 'deap' lib)
toolbox.register("mutate_uniform", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("mutate_node_replacement", gp.mutNodeReplacement, pset=pset)
toolbox.register("mutate_insert", gp.mutInsert,  pset=pset)
toolbox.register("mutate_hoist", mutHoist)

# Custom
toolbox.register("mutate_ephemeral", mutEphemeralByFitness, scale = 1e-2, mode='all')
toolbox.register("mutate_scale", mutScale, scale = 1e-2)
toolbox.register("mutate_add_const", mutAddConstant, pset=pset)

'''
  Mutation Operator |   Apply probability
--------------------|----------------------
 Uniform            |         20%
 Node replacemnet   |         20% 
 Ephemeral          |         30%
 Insert             |         10%
 Hoist              |         5%
 Add constant       |         5%
 Scale              |         10%
'''

toolbox.register("mutate",chooseMutate,
                operators = (toolbox.mutate_uniform,
                             toolbox.mutate_node_replacement,
                             toolbox.mutate_ephemeral,
                             toolbox.mutate_insert,
                             toolbox.mutate_hoist,
                             toolbox.mutate_add_const,
                             toolbox.mutate_scale),
                 probs = (0.2,0.2,0.3,0.1,0.05,0.05,0.1))
# toolbox.register("mutate",chooseMutate,
#                 operators = (toolbox.mutate_uniform,
#                              toolbox.mutate_node_replacement,
#                              toolbox.mutate_ephemeral,
#                              toolbox.mutate_insert,
#                              toolbox.mutate_hoist,
#                              toolbox.mutate_add_const,
#                              toolbox.mutate_scale),
#                  probs = (0.2,0.35,0.05,0.2,0.05,0.05,0.1))

## Migration

In [ ]:
toolbox.register("migrate",tools.migRing,k=20,selection=tools.selBest)

### Specify the height limits due to stack overflow error in algorithms

In [ ]:
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))

## Plotting

In [ ]:
from abc import ABC, abstractmethod
class PlotInteractive(ABC):
    def __init__(self, fig, ax):
        self.fig = fig
        self.ax = ax
    @abstractmethod
    def plot_approx(self, r_approx, title):
        pass

In [ ]:
class PlotInteractive2D(PlotInteractive):
    def __init__(self, fig, ax, thetas, r_exact_vals):
        super().__init__(fig, ax)
        self.thetas = thetas
        self.x = np.cos(self.thetas)
        self.y = np.sin(self.thetas)
        
        self.curve_exact = ax.plot(r_exact_vals*self.x, r_exact_vals*self.y, 'g-')
#         self.curve_approx = ax.plot(self.x, self.y, 'b--')
#         self.curve_approx_arr = [ax.plot(self.x, self.y, 'b--') for i in range(n_plot)]
        self.curve_approx_arr = [] 

    def plot_many(self, r_approx_arr, title):
        if len(self.curve_approx_arr) == 0:
            for r_vals in r_approx_arr:
                self.curve_approx_arr.append(ax.plot(r_vals*self.x,r_vals*self.y,'--'))
        else:
            for r_vals, curve_approx in zip(r_approx_arr, self.curve_approx_arr):
                curve_approx[0].set_xdata(r_vals*self.x)
                curve_approx[0].set_ydata(r_vals*self.y)
        
        self.ax.set_title(title)
        self.fig.canvas.draw()

    def plot_approx(self, r_approx, errors, title):
        
        self.curve_approx[0].set_xdata(r_approx*self.x)
        self.curve_approx[0].set_ydata(r_approx*self.y)
#         if self.plot_errors:
#             self.scatter_error.remove()
#             self.scatter_error = ax.scatter(r_approx*self.x, r_approx*self.y, s =20, c =errors  )
#             self.fig.colorbar(self.scatter_error, ax= self.ax)

        self.ax.set_title(title)
        self.fig.canvas.draw()

In [ ]:
class PlotInteractive3D(PlotInteractive):
    def __init__(self, fig, ax, thetas, phis, r_exact_vals):
        super().__init__(fig, ax)
        self.thetas = thetas
        self.phis = phis
        self.x = np.outer(np.cos(self.phis), np.sin(self.thetas))
        self.y = np.outer(np.sin(self.phis), np.sin(self.thetas))
        self.z = np.outer(np.ones(np.size(self.phis)), np.cos(self.thetas))
        self.surf_exact = ax.plot_wireframe(r_exact_vals*self.x,r_exact_vals*self.y,r_exact_vals*self.z, colors='lime')
        self.surf_approx = ax.plot_wireframe(self.x,self.y,self.z)

    def plot_approx(self, r_approx, errors, title):
        self.surf_approx.remove()
        self.surf_approx = self.ax.plot_wireframe(
            r_approx*self.x,
            r_approx*self.y,
            r_approx*self.z,
            colors='royalblue')
        self.ax.set_title(title)
        fig.canvas.draw()

In [ ]:
# %matplotlib notebook
# import matplotlib.pyplot as plt
# def plot_approximation(individual, title, plot_interactive):
#     r_vals = individual.eval_values
#     errors = abs(individual.error_values)
#     if DIM_2D:
#         plot_interactive.plot_approx(np.append(r_vals,r_vals[0]),np.append(errors,errors[0]), title)
#     else:
#         plot_interactive.plot_approx(r_vals,individual.error_values, title)

%matplotlib notebook
import matplotlib.pyplot as plt
def plot_approximation(individuals, title, plot_interactive):
    r_vals_arr = [np.append(ind.eval_values,ind.eval_values[0]) for ind in individuals]
    plot_interactive.plot_many(r_vals_arr, title)

In [ ]:
if DIM_2D:
    fig, ax = plt.subplots(1,1)
    ax.axis('equal')
    plt.ion()
    fig.show()
    fig.canvas.draw()
    plot_interactive_object = PlotInteractive2D(
        fig = fig,
        ax = ax,
        thetas = collocation.thetas_closed,
        r_exact_vals = R_EXACT(collocation.thetas_closed)
    )
else:
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    
    # TODO: set axis limits automaticly
    ax.set_xlim3d([-3, 3])
    ax.set_ylim3d([-3, 3])
    ax.set_zlim3d([-3, 3])
    plot_interactive_object = PlotInteractive3D(
        fig = fig,
        ax = ax,
        thetas = collocation.thetas,
        phis = collocation.phis_closed,
        r_exact_vals = R_EXACT(thetas_mesh, phis_mesh)
    )

%matplotlib inline
toolbox.register("plot", plot_approximation, plot_interactive = plot_interactive_object)

In [ ]:
def algorithm(pop_size, gen_num,
              statistics,
              initial_guesses = None,
              checkpoint_freq = 200,
              hof_size = 5):
    
    # Insert initial guesses if given
    if(initial_guesses is None):
        population = toolbox.population(n=pop_size)
    else:
        population = toolbox.population(n=pop_size-len(initial_guesses))
        population.extend(initial_guesses)
    
    # Saves best individual
    hof = tools.HallOfFame(hof_size)
    
    # Evaluate the entire population
    for ind in population:
        if not ind.fitness.valid:
            ind.fitness.values = toolbox.evaluate(ind)
    
    # Update hof
    hof.update(population)
    
    # Register logbook
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (statistics.fields if statistics else [])
    
    record = statistics.compile(population)
    logbook.record(gen=0, nevals=len(population), **record)
    
    # Print progress
    print(logbook.stream)
    
    for generation in range(1, gen_num+1):
        population = toolbox.execute_one(population)
        
        num_eval = 0
        # Evaluate the individuals with an invalid fitness
        for ind in population:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)
                num_eval +=1

        # Update hof
        hof.update(population)
        
        # Record and print progress
        record = statistics.compile(population)
        logbook.record(gen=generation, nevals=num_eval, **record)
        print(logbook.stream)
        
        # Plot current solution
        toolbox.plot(hof,title = f'Iteration {generation}, best fitness: {hof[0].fitness.values[0]}')
        
        if toolbox.stop_condition(logbook):
            break
    return population, logbook, hof

## Define execution model

In [ ]:
def rModel(population, cross_prob, mut_prob, rng):
    changeSize = int(0.4*len(population))
    
    # Select parents and copy them to offspring
    parents = toolbox.select_tournament(population, changeSize)

    offspring = [toolbox.clone(ind) for ind in parents]

    # Select died elements and remove them from population
    to_die = toolbox.select_worst(population, changeSize)

    population = [ind for ind in population if not any(ind is copy for copy in to_die)]

    # Apply crossover and mutation to create offspring
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if rng.random() < cross_prob:
            ind1,ind2 = toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values

    for ind in offspring:
        if rng.random() < mut_prob:
            ind, = toolbox.mutate(ind)
            del ind.fitness.values

    # Extend population by offspring
    population.extend(offspring)
    
    return population

In [ ]:
def simpleModel(population, cross_prob, mut_prob, rng):    
    # Select parents and copy them to offspring
    parents = toolbox.select_tournament(population, len(population))
    offspring = [toolbox.clone(ind) for ind in parents]

    # Apply crossover and mutation to create offspring
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if rng.random() < cross_prob:
            ind1,ind2 = toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values

    for ind in offspring:
        if rng.random() < mut_prob:
            ind, = toolbox.mutate(ind)
            del ind.fitness.values
    
    return offspring

In [ ]:
def muPlusLambda(population, cross_prob, mut_prob, mu, lambda_, rng):
    offspring = []
    for _ in range(lambda_):
        if rng.random() < cross_prob:
            ind1, ind2 = [toolbox.clone(i) for i in random.sample(population, 2)]
            ind1, ind2 = toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            offspring.append(ind1)
        elif rng.random() < mut_prob:
            ind = toolbox.clone(random.choice(population))
            ind, = toolbox.mutate(ind)
            del ind.fitness.values
            offspring.append(ind)
        else:
            offspring.append(random.choice(population))
    population[:] = toolbox.select_tournament(population + offspring, mu)
    
    return population

In [ ]:
def simpleStopCond(logbook, max_no_imporve_iters):
    fitnesses = logbook.select("min")
    
    if len(fitnesses) > max_no_imporve_iters:
        return fitnesses[-1] > fitnesses[-max_no_imporve_iters]

    return False

def deltaStopCond(logbook, max_no_imporve_iters,delta):
    fitnesses = logbook.select("min")
    
    if len(fitnesses) > max_no_imporve_iters:
        return abs(fitnesses[-1] - fitnesses[-max_no_imporve_iters]) < delta

    return False
    

In [ ]:
def algorightmWithGuesses(pop_size, gen_num, guess_pop_size, guess_gen_num, stats, n_guesses):
    best_solutions = []
    
    for i in range(n_guesses):
        print(f'Guess {i+1}/{n_guesses} {"-"*100}')
        _, _, curr_hof = algorithm(pop_size = guess_pop_size, gen_num = guess_gen_num, statistics = stats)
        best_solutions.extend(curr_hof)
    
    
    return algorithm(
        pop_size = pop_size + len(best_solutions),
        gen_num = gen_num,
        statistics = stats, 
        initial_guesses = best_solutions)


In [ ]:
# Define statistics
stats = tools.Statistics(lambda ind: ind.fitness.values)

stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [ ]:
toolbox.register("execute_one", rModel, cross_prob = 0.65, mut_prob = 0.25, rng = np.random.default_rng())
# toolbox.register("execute_one", simpleModel, cross_prob = 0.65, mut_prob = 0.3, rng = np.random.default_rng())
#toolbox.register("execute_one", muPlusLambda, cross_prob = 0.65, mut_prob = 0.45, mu = 1000, lambda_ = 500, rng = np.random.default_rng())

In [ ]:
# toolbox.register("stop_condition", simpleStopCond, max_no_imporve_iters = 100)
toolbox.register("stop_condition", deltaStopCond, max_no_imporve_iters = 150, delta=1e-5)

In [ ]:
population, logbook, hof = algorightmWithGuesses(pop_size = 500,
                      gen_num = 500, 
                      guess_pop_size = 200,
                      guess_gen_num = 200,
                      stats = stats, 
                      n_guesses = 10)

In [ ]:
for ind in hof:
    print(ind,'\n\n')